# Compute grads and save to file

In [1]:
import os
import sys
sys.path.insert(0, '/home/jovyan/braindecode/')
import pickle

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from decode_tueg import (
    load_exp, DataScaler, TargetScaler, Augmenter, 
    save_csv, iter_exp_dir, get_freqs_and_info, compute_gradients,
)

Wed Feb  8 14:37:48 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 515.48.07    Driver Version: 515.48.07    CUDA Version: 11.7     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA GeForce ...  On   | 00000000:89:00.0 Off |                  N/A |
| 27%   27C    P8     8W / 250W |   1582MiB / 11264MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
batch_size = 64
n_jobs = 4
n_recordings = 100
exp_dir = '/home/jovyan/experiments/'
exp_date = '2023-02-07T06:28:03.373666'
checkpoint = 'train_end'

In [3]:
# TODO: compute on eval, too
for ds_name in ['transition', 'non_pathological', 'pathological']:
#    seeds = sorted(os.listdir(os.path.join(exp_dir, exp_date)))
#    for seed in seeds:
#        runs = sorted(os.listdir(os.path.join(exp_dir, exp_date, seed)))
#        for run_i, run in enumerate(runs):
#            exp_path = os.path.join(exp_date, seed, run)
    for exp_path in iter_exp_dir(exp_dir, exp_date):
        print(f'loading exp {exp_path}')
        clf, data_scaler, target_scaler, config = load_exp(
            exp_dir, exp_path, checkpoint)
        print(f'getting {ds_name}')
        with open(f'/home/jovyan/longitudinal/{ds_name}_pre_win.pkl', 'rb') as f:
            ds = pickle.load(f)
        if n_recordings not in [-1, None]:
            ds = ds.split(list(range(n_recordings)))['0']

        print('computing gradients')
        # compute grads and freqs
        grads = compute_gradients(clf, ds, batch_size, n_jobs)
        freqs, info = get_freqs_and_info(ds)

        # create grads output subdir
        out_path = os.path.join(exp_dir, exp_path, 'grads')
        print(f'saving {out_path}')
        if not os.path.exists(out_path):
            os.makedirs(out_path)

        # save grads to csv file
        for k, v in grads.items():
            grads_df = pd.DataFrame(v, index=info.ch_names, columns=freqs)
            save_csv(grads_df, out_path, f'{k}.csv')
        break
    break

loading exp 2023-02-07T06:28:03.373666/20230206/0
getting transition
computing gradients


NameError: name 'make_standard_montage' is not defined

# Read grads from files and plot

In [6]:
import os
import sys
sys.path.insert(0, '/home/jovyan/braindecode/')
import pandas as pd
from decode_tueg import (
    plot_band_grads, freqs_to_bin, iter_exp_dir)

In [7]:
exp_dir = '/home/jovyan/experiments/'
exp_date = '2023-02-07T06:28:03.373666'
checkpoint = 'train_end'

In [8]:
for exp_path in iter_exp_dir(exp_dir, exp_date):
    break

In [9]:
exp_path

'2023-02-07T06:28:03.373666/20230206/0'

In [12]:
for ds_name in ['longitudinal_transition', 'longitudinal_non_patholotigal', 'longitudinal_pathological']:  # eval
    break

In [13]:
pd.read_csv(os.path.join(exp_dir, exp_path, 'grads', f'{checkpoint}_{ds_name}_preds.csv'), index_col=0)

FileNotFoundError: [Errno 2] No such file or directory: '/home/jovyan/experiments/2023-02-07T06:28:03.373666/20230206/0/grads/train_end_longitudinal_transition_preds.csv'

In [3]:
bands = [(0,4),(4,8),(8,13),(13,30),(30,50)]

In [4]:
save = False

In [5]:
for band in bands:
    all_band_grads = {}
    for key, g in grads.items():
        # compute all bands gradients
        l, h = freqs_to_bin(freqs, band)
        band_freqs = freqs[l:h+1]
        band_grads = g[:,l:h+1].mean(axis=1)
        all_band_grads[key] = band_grads        

    fig = plot_band_grads(all_band_grads, info)
    if save:
        fig.savefig()

NameError: name 'grads' is not defined